In [1]:
import re
import qiskit
import json
import time

from qiskit import transpile
import time
import os

from fast_generator import fc_tree_commute_recur_lookahead_fast
from circuit_generator import construct_qcc_circuit
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers

In [2]:
#Compare json files in a specific folder
def run_experiment_folder(folder_path = None, filename = None, save_output = False, threshold = 1):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            combined_time = end_time - start_time
            
            # Measure time for Qiskit method
            start_time = time.time()
            origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
            origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            qiskit_time = end_time - start_time
            
            # # Measure time for RustiQ method
            # start_time = time.time()
            # circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
            # end_time = time.time()
            # rustiq_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time,
                    "qiskit_time": qiskit_time,
                    # "rustiq_time": rustiq_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    # "rustiq_method": entangling_count(circuit)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    # "rustiq_method": entangling_depth(circuit)
                },
                "test_paulis_file": f'benchmarks/results/test_new_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_new_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [3]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, threshold = 1, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time

    # # Measure time for RustiQ method
    # start_time = time.time()
    # circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    # end_time = time.time()
    # rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            # "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            # "rustiq_method": entangling_count(circuit)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            # "rustiq_method": entangling_depth(circuit)
        },
        "test_paulis_file": f'benchmarks/results/test_new_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_new_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [ ]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20,]# 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = False, threshold = 1, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.07702898979187012, 'combined_time': 0.1219949722290039, 'qiskit_time': 0.08829116821289062}, 'gate_counts': {'our_method': 27, 'combined_method': 23, 'qiskit_method': 35}, 'circuit_entangling_depth': {'our_method': 21, 'combined_method': 17, 'qiskit_method': 35}, 'test_paulis_file': 'benchmarks/results/test_new_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 0.3335411548614502, 'combined_time': 0.4408597946166992, 'qiskit_time': 0.340487003326416}, 'gate_counts': {'our_method': 119, 'combined_method': 106, 'qiskit_method': 159}, 'circuit_entangling_depth': {'our_method': 94, 'combined_method': 82, 'qiskit_method': 156}, 'test_paulis_file': 'benchmarks/results/test_new_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 1.8723409175872803, 'combined_time': 2.8864481449127197, 'qiskit_time': 2.0555148124694824}, 'gate_counts': {'our_method': 507, 'combined_method': 448, 'qiskit_method': 760}, 'circuit_entangling_depth': {'our_meth

In [ ]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = False, threshold = 1)

In [ ]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = False, threshold = 1)

In [ ]:
#since in the maxcut problems there are many commuting terms and it's slow to search through all of them,
#we can set the threshold to 2 to early stop the search once found a simplified pauli of weight 2
#you can see the run time difference, for the second benchmark runtime reduces from 30second to 7 seconds.

# run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 2)
all_filenames=["max_cut_benchmark_regular_n20_e4_l1.json", "max_cut_benchmark_regular_n20_e4_l3.json", 
               "max_cut_benchmark_regular_n20_e8_l1.json", "max_cut_benchmark_regular_n20_e8_l3.json",
               "max_cut_benchmark_regular_n20_e12_l1.json", "max_cut_benchmark_regular_n20_e12_l3.json"]
for filename in all_filenames:
    run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", filename=filename, save_output = False, threshold = 2)

In [ ]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = False, threshold= 2)